In [ ]:
import os
import pickle
import sparse
import librosa
import mir_eval
import fluidsynth
import pretty_midi
import numpy as np
import IPython.display

from glob import glob
from utils.drum_utils import *
from utils.common_utils import *

%load_ext autoreload
%autoreload 2

### Preprocessing

In [ ]:
data = []
count = 0 # num of bars
data_path = '/data/Lakh/'
save_path = '/data/Lakh_processed/Lakh_9drum_16bar_4of4_16quant.pkl'

In [ ]:
for folder_path in glob(data_path+'*'):
    for file_path in glob(folder_path+'/*'):
        try:
            pm = pretty_midi.PrettyMIDI(file_path)

            # time signature 4/4 check
            if not check_time_sign(pm, num=4, denom=4): 
                continue

            start_time = pm.get_onsets()[0]
            beats = pm.get_beats(start_time)
            tempo = pm.estimate_tempo()
            fs = change_fs(beats)

            # for each inst
            for inst in pm.instruments:
                if inst.is_drum == True:
                    drum_roll = quantize_drum(inst, fs, start_time)
                    drum_roll = windowing(drum_roll)
                    drum_roll = hot_encoding(drum_roll)
                    
                    for i in range(0, drum_roll.shape[0]):
                        # to reduce size of data, sparse encoding
                        data.append(sparse.COO(drum_roll[i]))
        except:
            continue
                    
        # verbose
        count += 1
        if count % 100 == 0:
            print('Files iterations %d' % count)

In [ ]:
# play music example
fs = 10
pm = drum_play(data[100].todense().astype('float32'), fs)
IPython.display.Audio(pm.fluidsynth(fs=16000), rate=16000)

In [ ]:
# save pickle
with open(save_path, 'wb') as f:
    print('File saved!')
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

# laod pickle
with open(save_path + '.pkl', 'rb') as f:
    a = pickle.load(f)